In [1]:
import sys
sys.modules[__name__].__dict__.clear()
import pandas as pd
import yaml
import numpy as np
import os
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.linalg import svd

In [2]:
#read data and windowing settings from YML
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
file_name = cfg['filepath']
window_size = cfg['window_size']
overlap = cfg['overlap']
num_clusters = cfg['num_clusters'] 
data  = pd.read_csv(file_name)
sort_along = cfg['slide_along'] #bydefault set to east. go to config file if want to slide along other directions.

In [3]:
print(data.shape)
#split into train and test 
train_data = data[data["dhid"]<500]

(66635, 22)


In [4]:
#seperating variable and categorical variables for different distance measure 
train_data_grades =  train_data[['midx','midy','midz','cut_pc','cus_pc','co3_pc','no3_pc','au_ppm','mo_ppm','pb_ppm','ag_ppm','as_ppm']]
train_data_categorical =  train_data[['midx','midy','midz','lito_m','alte_m','zmin_m','lito_r','eu_pro','str_zone']]
print(train_data_grades.head())
print(train_data_categorical.head())

     midx    midy   midz  cut_pc  cus_pc  co3_pc  no3_pc  au_ppm  mo_ppm  \
0  3550.1  1490.6  897.5   0.006   -99.0   -99.0   -99.0   -99.0   -99.0   
1  3549.8  1490.1  895.5   0.006   -99.0   -99.0   -99.0   -99.0   -99.0   
2  3549.5  1489.7  893.6   0.008   -99.0   -99.0   -99.0   -99.0   -99.0   
3  3549.2  1489.3  891.7   0.003   -99.0   -99.0   -99.0   -99.0   -99.0   
4  3548.8  1488.9  889.8   0.004   -99.0   -99.0   -99.0   -99.0   -99.0   

   pb_ppm  ag_ppm  as_ppm  
0     -99   -99.0     -99  
1     -99   -99.0     -99  
2     -99   -99.0     -99  
3     -99   -99.0     -99  
4     -99   -99.0     -99  
     midx    midy   midz  lito_m  alte_m  zmin_m  lito_r  eu_pro  str_zone
0  3550.1  1490.6  897.5     500     800     100       2       1         5
1  3549.8  1490.1  895.5     500     800     100       2       1         5
2  3549.5  1489.7  893.6     500     800     100       2       1         5
3  3549.2  1489.3  891.7     500     800     100       2       1         5


In [6]:
#from scipy.linalg import svd
#U, s, VT = svd(train_data_grades.iloc[:20,])

In [6]:
'''This is an example of converting a grid raw data to a feature vector
kurt = train_data_grades.kurtosis(axis = 0)
#print(kurt)
skewness = train_data_grades.skew(axis = 0)
mean = train_data_grades.mean(axis = 0)
std_dev = train_data_grades.std(axis = 0)
df = np.matrix(pd.concat([kurt,skewness,mean,std_dev],axis = 1))
#fuse matrix to 1 D array
df_vec = df.getA1()
#append top 3 singular values
U, s, VT = svd(train_data_grades.iloc[:20,])
#top 3 singular values
top3_sv = s[0:3]
feature_vec=np.append(df_vec,top3_sv)
print(feature_vec)
'''

[ 4.20412810e-01  2.13562053e-01  1.73650161e+03  7.18684615e+02
  1.35723253e+00  5.13426529e-01  1.69795415e+03  4.92282293e+02
  9.15796308e-01 -1.05310419e+00  6.57344662e+02  1.40623907e+02
  6.69409300e+00 -2.94600605e+00 -8.32671602e+00  2.78429269e+01
 -1.44157990e+00  7.46187613e-01 -6.66779896e+01  4.65426722e+01
  2.99589974e-03  1.41432111e+00 -7.77424171e+01  4.10506083e+01
  7.75657073e-03  1.41695181e+00 -7.80966993e+01  4.04269486e+01
 -1.39066251e+00 -7.80615990e-01 -3.14701214e+01  4.61345199e+01
  7.36907428e+01  4.58389552e+00  1.47915684e+01  1.01505817e+02
  9.09249805e+01  7.08632307e+00 -9.43848978e+01  2.69272978e+01
  2.82325906e+01  5.49778958e+00 -9.59922894e+01  1.70630289e+01
  0.00000000e+00  0.00000000e+00 -9.90000000e+01  0.00000000e+00
  1.76859252e+04  1.60472364e+02  3.57824730e+01]


In [9]:
'''Import windows slider class'''
from Window_slide import Window_slide 
# Defining Function to extract features: calculates kurtosis, mean, skewness, st dev of different attributtes
def extract_features(window_matrix):    
    kurt = window_matrix.kurtosis(axis = 0)
    skewness = window_matrix.skew(axis = 0)
    mean = window_matrix.mean(axis = 0)
    std_dev = window_matrix.std(axis = 0)
    df = np.matrix(pd.concat([kurt,skewness,mean,std_dev],axis = 1))
#fuse matrix to 1 D array
    df_vec = df.getA1()
#append top 3 singular values
    U, s, VT = svd(train_data_grades.iloc[:20,])
#top 3 singular values
    top3_sv = s[0:3]
    feature_vec=np.append(df_vec,top3_sv)
    return(feature_vec)


In [8]:
def hamming_distance(s1, s2):
    assert len(s1) == len(s2)
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

#example of hamming distance:
import numpy as np
x = np.array([1,2,3,7])
y = np.array([0,1,1,3])
print("hamming distance between ", x, "and ",y , "is" ,  hamming_distance(x,y))

hamming distance between  [1 2 3 7] and  [0 1 1 3] is 4


In [10]:
'''Dealing with variable data (grades, etc)
1. Window slides along the direction specified in config file
2. function call for each block to calculate features and append to data frame
3. End when last window readched
'''
#Data should be sorted in one direction either x,y,z
sorted_data = train_data_grades.sort_values(sort_along)
#initialise class : window
window = Window_slide(sorted_data,window_size,overlap)
features_single_block = pd.DataFrame();
while True:
    window_matrix = window.slide()
    if len(window_matrix ==window_size):
        #print("TO BE DEVELOPED ...extracting features for block i .... Printing data in block" ,window_matrix )
        features_single_block = features_single_block.append(extract_features(window_matrix), ignore_index=True)
        if window.reached_end_of_list(): break

TypeError: cannot concatenate object of type "<class 'numpy.ndarray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [11]:
print(window_matrix)

      midx    midy   midz  cut_pc  cus_pc  co3_pc  no3_pc  au_ppm  mo_ppm  \
1435  44.7  1435.7  592.3   0.099 -99.000 -99.000   -99.0   0.005     5.0   
1434  45.1  1435.4  594.3   0.101 -99.000 -99.000   -99.0   0.005     5.0   
1433  45.6  1435.1  596.2   0.097 -99.000 -99.000   -99.0   0.005     5.0   
1432  46.0  1434.9  598.1   0.082 -99.000 -99.000   -99.0   0.005     5.0   
1431  46.4  1434.6  600.1   0.041 -99.000 -99.000   -99.0   0.005    11.0   
1430  46.8  1434.4  602.0   0.043 -99.000 -99.000   -99.0   0.005    26.0   
1429  47.3  1434.1  604.0   0.098 -99.000 -99.000   -99.0   0.005    32.0   
1428  47.7  1433.9  605.9   0.124 -99.000 -99.000   -99.0   0.005    21.0   
1427  48.1  1433.6  607.8   0.156 -99.000 -99.000   -99.0   0.005     5.0   
1426  48.5  1433.3  609.8   0.114 -99.000 -99.000   -99.0   0.005     5.0   
1425  49.0  1433.1  611.7   0.200 -99.000 -99.000   -99.0   0.005     5.0   
1424  49.4  1432.8  613.6   0.258 -99.000 -99.000   -99.0   0.005    15.0   

In [ ]:
#Discard rows again with missing freatures
feature_set = pd.DataFrame(features_single_block.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False))
print(feature_set.head())